# RestaurarIA

In [1]:
import os

#### Check GPU

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

#### Clone Dependencies

In [ ]:
!mkdir src && mkdir frames

In [ ]:
%cd src

In [ ]:
!wget https://github.com/nihui/rife-ncnn-vulkan/releases/download/20210520/rife-ncnn-vulkan-20210520-ubuntu.zip #RIFE

In [ ]:
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20210521/waifu2x-ncnn-vulkan-20210521-ubuntu.zip #WAIFU2X

In [ ]:
!wget https://github.com/nihui/realsr-ncnn-vulkan/releases/download/20210210/realsr-ncnn-vulkan-20210210-ubuntu.zip #REALSR

In [ ]:
!unzip rife-ncnn-vulkan-20210520-ubuntu.zip && rm rife-ncnn-vulkan-20210520-ubuntu.zip

In [ ]:
!unzip waifu2x-ncnn-vulkan-20210521-ubuntu.zip && rm waifu2x-ncnn-vulkan-20210521-ubuntu.zip

In [ ]:
!unzip realsr-ncnn-vulkan-20210210-ubuntu.zip && rm realsr-ncnn-vulkan-20210210-ubuntu.zip

In [ ]:
%cd ..

#### Install 

In [ ]:
!apt-get update && apt-get install ffmpeg

#### Config

In [ ]:
################# Required Configurations ############################

inputfile = "inspector.mp4" #param{type:"string"}

outputfile = "inspectorRest.mp4" #param{type:"string"}

In [ ]:
################# EX Config ############################

RIFE = "./src/rife-ncnn-vulkan-20210520-ubuntu/rife-ncnn-vulkan" #param{type:"string"}
WAIFU2X = "./src/waifu2x-ncnn-vulkan-20210521-ubuntu/waifu2x-ncnn-vulkan" #param{type:"string"}
REALSR = "./src/realsr-ncnn-vulkan-20210210-ubuntu/realsr-ncnn-vulkan" #param{type:"string"}


In [ ]:
################# Aux paths ############################

Original_Frames = 'frames/Original' #@param{type:"string"}

HD_Frames = 'frames/HD' #@param{type:"string"}

HFR_Frames = 'frames/HFR' #@param{type:"string"}

In [ ]:
####### TELEGRAM SETTINGS 

from telegram_notifier import set_config_options
from telegram_notifier import Notifier

TOKEN = '1661115267:AAFQ6ED6lZjKkedrk-uD50GIZUfry-Y_iLg'
MYID = '45762408'

set_config_options(chat_id=MYID, token=TOKEN)

#### Check Video Settings

In [ ]:
!ffprobe '{inputfile}'

#### Extract Audio

In [ ]:
!ffmpeg -i '{inputfile}' -vn -acodec copy audio.m4a

#### Extract Frames

In [ ]:
rm -R '{Original_Frames}' && rm -R '{HD_Frames}' && rm -R '{HFR_Frames}'

In [ ]:
mkdir '{Original_Frames}' && mkdir '{HD_Frames}' && mkdir '{HFR_Frames}'

In [ ]:
with Notifier("Extracting Frames"):
    !ffmpeg -r 24 -i '{inputfile}' '{Original_Frames}/%08d.png'
    pass

In [ ]:
numogf = len([name for name in os.listdir('Original_Frames') if os.path.isfile(name)])

#### Processing Video

In [ ]:
with Notifier("REALSR"):
    !'{REALSR}' -i '{HFR_Frames}' -o '{HD_Frames}' -s 4 -j 4:4:4 -m models-DF2K -x
    pass

In [ ]:
with Notifier("WAIFU2X"):
    !'{WAIFU2X}' -i '{HFR_Frames}' -o '{HD_Frames}' -s 4 -n 0 -j 4:4:4 -x
    pass

In [ ]:
with Notifier("RIFE"):
    !'{RIFE}' -i '{Original_Frames}' -o '{HFR_Frames}' -j 4:4:4 -m rife-v3.1 -u -x 
    pass

#### Encode Video

In [ ]:
with Notifier("Encoding Video"):
    !ffmpeg -framerate 48 -i frames/HD/%08d.png -i audio.m4a -c:a copy -crf 20 -c:v libx264 -pix_fmt yuv420p '{outputfile}'
    pass